In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
import pickle
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [2]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
delimiters=["।","?","!"]

In [4]:
file=open('glove.pkl','rb')
glove=pickle.load(file)
file.close()

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [8]:
def word_embedding(word):
    try:
        return glove[word]
    except:
        return np.array([0.0]*100)

In [9]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [10]:
def ed(a,b):
    return lin.norm(a-b)

In [11]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [12]:
data=pd.read_csv('../hindi.csv')

In [13]:
query='भौतिक जगत में माया का आधिपत्य होता है और श्रीकृष्ण ने इसे आठवें अध्याय के 15वें श्लोक में अस्थायी और दुखों से भरा हुआ कह कर परिभाषित किया है। इस प्रकार भौतिक शक्ति माया की तुलना अंधकार से की जाती है।'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [14]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [15]:
verse_index=max_cosine(query_embedding)

In [16]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
57,2,16,नासतो विद्यते भावो नाभावो विद्यते सतः।\n\nउभयो...,श्वेताश्वतरोपनिषद् के अनुसार संसार में तीन तत्...
518,14,1,श्री भगवानुवाचपरं भूयः प्रवक्ष्यामि ज्ञानानां ...,पिछले अध्याय में श्रीकृष्ण ने यह व्यक्त किया थ...
229,6,2,यं संन्यासमिति प्राहुर्योगं तं विद्धि पाण्डव।\...,संन्यासी वही है जो मन और इन्द्रियों के सुखों क...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
557,15,13,गामाविश्य च भूतानि धारयाम्यहमोजसा।पुष्णामि चौष...,'गाम' शब्द का अर्थ पृथ्वी और 'ओजसा' शब्द का अर...
418,11,10,अनेकवक्त्रनयनमनेकाद्भुतदर्शनम्।\n\nअनेकदिव्याभ...,संजय अनेक और अनंत शब्दों का प्रयोग कर श्रीकृष...
419,11,11,दिव्यमाल्याम्बरधरं दिव्यगन्धानुलेपनम्।\n\nसर्व...,संजय अनेक और अनंत शब्दों का प्रयोग कर श्रीकृष...
416,11,8,न तु मां शक्यसे द्रष्टुमनेनैव स्वचक्षुषा।\n\nद...,जब परमप्रभु संसार में अवतरित होते हैं तब उनके ...
502,13,20,प्रकृतिं पुरुषं चैव विद्ध्यनादी उभावपि।विकारां...,भौतिक शक्ति को माया या प्रकृति कहा जाता है। भग...


## Euclidean Distance

In [17]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [18]:
verse_index=max_ed(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
654,18,38,विषयेन्द्रियसंयोगाद्यत्तदग्रेऽमृतोपमम्।परिणामे...,राजसिक सुख एक रोमांच की भांति है जो इन्द्रियों...
21,1,25,भीष्मद्रोणप्रमुखतः सर्वेषां च महीक्षिताम्।\n\n...,"""कुरु"" शब्द का प्रयोग कौरवों और पाण्डवों दोनो..."
217,5,19,इहैव तैर्जितः सर्गो येषां साम्ये स्थितं मनः।\n...,श्रीकृष्ण ने यहाँ 'सम' शब्द का प्रयोग किया है...
518,14,1,श्री भगवानुवाचपरं भूयः प्रवक्ष्यामि ज्ञानानां ...,पिछले अध्याय में श्रीकृष्ण ने यह व्यक्त किया थ...
483,12,20,ये तु धर्म्यामृतमिदं यथोक्तं पर्युपासते।श्रद्द...,श्रीकृष्ण अब अर्जुन के प्रश्न की सारगर्भित व्य...
557,15,13,गामाविश्य च भूतानि धारयाम्यहमोजसा।पुष्णामि चौष...,'गाम' शब्द का अर्थ पृथ्वी और 'ओजसा' शब्द का अर...
160,4,4,अर्जुन उवाच\n\nअपरं भवतो जन्म परं जन्म विवस्वत...,अर्जुन भगवान श्रीकृष्ण के कथनों में घटनाओं की ...
474,12,11,अथैतदप्यशक्तोऽसि कर्तुं मद्योगमाश्रितः।सर्वकर्...,इस अध्याय के 8वें श्लोक का आरम्भ करने के साथ श...
64,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"चेतना, जो आत्मा का लक्षण है, उसे भौतिक अवयवों..."


# Mean Pooling

## Cosine Similarity

In [20]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [21]:
verse_index=mean_cosine(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
656,18,40,न तदस्ति पृथिव्यां वा दिवि देवेषु वा पुनः।सत्त...,श्वेताश्वतरोपनिषद् में माया शक्ति का विस्तारित...
287,7,13,त्रिभिर्गुणमयैर्भावैरेभिः सर्वमिदं जगत्।\n\nमो...,पिछले श्लोकों को सुनकर अर्जुन यह सोचता होगा “ह...
635,18,19,ज्ञानं कर्म च कर्ता च त्रिधैव गुणभेदतः।प्रोच्य...,श्रीकृष्ण एक बार पुनः प्रकृति के तीन गुणों का ...


## Euclidean Distance

In [23]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [24]:
verse_index=mean_ed(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
656,18,40,न तदस्ति पृथिव्यां वा दिवि देवेषु वा पुनः।सत्त...,श्वेताश्वतरोपनिषद् में माया शक्ति का विस्तारित...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
217,5,19,इहैव तैर्जितः सर्गो येषां साम्ये स्थितं मनः।\n...,श्रीकृष्ण ने यहाँ 'सम' शब्द का प्रयोग किया है...
506,13,24,य एवं वेत्ति पुरुषं प्रकृतिं च गुणैःसह।सर्वथा ...,अज्ञानता आत्मा को दुर्दशा की ओर ले जाती है। भग...
192,4,36,अपि चेदसि पापेभ्यः सर्वेभ्यः पापकृत्तमः।\n\nसर...,मायाबद्ध संसार गहरे समुद्र की भांति है जिसमें ...


# Line Matching

## Cosine Similarity

In [26]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [27]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
408,10,42,अथवा बहुनैतेन किं ज्ञातेन तवार्जुन।\n\nविष्टभ्...,श्रीकृष्ण का उपर्युक्त कथन यह इंगित करता है कि...
337,9,5,न च मत्स्थानि भूतानि पश्य मे योगमैश्वरम्।\n\nभ...,पिछले श्लोक में कथित रूप से उल्लिखित दो शक्ति...
678,18,62,तमेव शरणं गच्छ सर्वभावेन भारत।तत्प्रसादात्परां...,भगवान पर निर्भरता के कारण आत्मा को अपनी वर्तमा...
416,11,8,न तु मां शक्यसे द्रष्टुमनेनैव स्वचक्षुषा।\n\nद...,जब परमप्रभु संसार में अवतरित होते हैं तब उनके ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
129,3,16,एवं प्रवर्तितं चक्रं नानुवर्तयतीह यः।\n\nअघायु...,चक्र या चरण से तात्पर्य घटनाओं की क्रमबद्धता स...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...


## Euclidean Distance

In [28]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [29]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
430,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,ये सभी महान विभूतियाँ भगवान की शक्ति से ही अप...
160,4,4,अर्जुन उवाच\n\nअपरं भवतो जन्म परं जन्म विवस्वत...,अर्जुन भगवान श्रीकृष्ण के कथनों में घटनाओं की ...
37,1,41,अधर्माभिभवात्कृष्ण प्रदुष्यन्ति कुलस्त्रियः।\n...,वैदिक सभ्यता में समाज में स्त्रियों को उच्च स...
64,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"चेतना, जो आत्मा का लक्षण है, उसे भौतिक अवयवों..."
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...
654,18,38,विषयेन्द्रियसंयोगाद्यत्तदग्रेऽमृतोपमम्।परिणामे...,राजसिक सुख एक रोमांच की भांति है जो इन्द्रियों...
519,14,2,इदं ज्ञानमुपाश्रित्य मम साधर्म्यमागताः।सर्गेऽप...,श्रीकृष्ण अर्जुन को आश्वस्त करते हैं कि वे जो ...
474,12,11,अथैतदप्यशक्तोऽसि कर्तुं मद्योगमाश्रितः।सर्वकर्...,इस अध्याय के 8वें श्लोक का आरम्भ करने के साथ श...
429,11,21,अमी हि त्वां सुरसङ्घाः विशन्ति\n\nकेचिद्भीताः ...,अर्जुन ने यहाँ श्रीकृष्ण के कालरूप को देखा अर्...
29,1,33,येषामर्थे काङ्क्षितं नो राज्यं भोगाः सुखानि च।...,अर्जुन के मन में यह सत्य जानकर उलझन उत्पन्न हु...


# Whole Part

## Cosine Similarity

In [30]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [31]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
656,18,40,न तदस्ति पृथिव्यां वा दिवि देवेषु वा पुनः।सत्त...,श्वेताश्वतरोपनिषद् में माया शक्ति का विस्तारित...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
307,8,3,श्री भगवानुवाच\n\nअक्षरं ब्रह्म परमं स्वभावोऽध...,श्रीकृष्ण कहते हैं कि परम सत्ता को ब्रह्म कहा...


## Euclidean Distance

In [32]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [33]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
22,1,26,तत्रापश्यत्स्थितान्पार्थः पितृ़नथ पितामहान्।\n...,अर्जुन युद्धभूमि में अपने सभी सम्बंधियों को दे...
486,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,श्रीकृष्ण अब स्वयं कई प्रश्न खड़े करते हैं और ...
10,1,13,ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः।\n\nसहसैवा...,युद्ध के लिए भीष्म पितामह के तीव्र उत्साह को द...
597,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,"जब शाकाहारी भोजनों को अत्यधिक मीर्च, शर्करा, न..."
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।\n\nयुयु...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
4,1,5,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्।\n\nप...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
5,1,6,युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान्।\...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,दुर्योधन यह समझ गया था कि भीष्म पितामह मानसिक ...
660,18,44,कृषिगौरक्ष्यवाणिज्यं वैश्यकर्म स्वभावजम्।परिचर...,वैश्यों में रजोगुण की प्रधानता के साथ तमोगुण म...
598,17,10,यातयामं गतरसं पूति पर्युषितं च यत्।उच्छिष्टमपि...,ऐसे पके हुए भोज्य पदार्थ जिन्हें पकाये हुए एक ...


# Clustering

## Max Pooling

In [34]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [35]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [36]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
37,1,41,अधर्माभिभवात्कृष्ण प्रदुष्यन्ति कुलस्त्रियः।\n...,वैदिक सभ्यता में समाज में स्त्रियों को उच्च स...
38,1,43,दोषैरेतैः कुलघ्नानां वर्णसङ्करकारकैः।\n\nउत्सा...,सनातन-धर्म या वर्णाश्रम द्वारा निर्धारित मानव ...
44,2,3,क्लैब्यं मा स्म गमः पार्थ नैतत्त्वय्युपपद्यते।...,ज्ञानोदय के मार्ग पर सफलतापूर्वक कदम रखने के ल...
75,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम्।\n...,सम्मानित लोगों के लिए सामाजिक प्रतिष्ठा अति मह...
86,2,45,त्रैगुण्यविषया वेदा निस्त्रैगुण्यो भवार्जुन।\n...,मायाशक्ति अपने तीन प्रकार के प्राकृतिक गुणों द...
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
129,3,16,एवं प्रवर्तितं चक्रं नानुवर्तयतीह यः।\n\nअघायु...,चक्र या चरण से तात्पर्य घटनाओं की क्रमबद्धता स...
188,4,32,एवं बहुविधा यज्ञा वितता ब्रह्मणो मुखे।\n\nकर्म...,वेदों की एक सुन्दर विशेषता यह है कि वे मनुष्यो...
273,6,46,तपस्विभ्योऽधिको योगी ज्ञानिभ्योऽपि मतोऽधिकः।\n...,तपस्वी वह है जो स्वेच्छा से वैराग्य स्वीकार कर...
285,7,11,बलं बलवतां चाहं कामरागविवर्जितम्।\n\nधर्माविरु...,आसक्ति-अप्राप्त सुख सुविधाओं के लिए तीव्र उत्क...


## Mean Pooling

In [37]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [38]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [39]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
61,2,20,न जायते म्रियते वा कदाचि\n\nन्नायं भूत्वा भवित...,इस श्लोक में आत्मा की शाश्वतता को सिद्ध किया ग...
69,2,28,अव्यक्तादीनि भूतानि व्यक्तमध्यानि भारत।\n\nअव्...,इस अध्याय के श्लोक 2.20 में श्रीकृष्ण ने आत्मा...
278,7,4,भूमिरापोऽनलो वायुः खं मनो बुद्धिरेव च।\n\nअहङ्...,"प्राकृत शक्ति से रचित संसार विचित्र, जटिल और अ..."
386,10,20,अहमात्मा गुडाकेश सर्वभूताशयस्थितः।\n\nअहमादिश्...,श्रीकृष्ण स्पष्ट कहते हैं कि वे आत्मा से दूर न...
405,10,39,यच्चापि सर्वभूतानां बीजं तदहमर्जुन।\n\nन तदस्त...,श्रीकृष्ण समस्त सृष्टि के कुशल कारण और उपादान...
516,13,34,यथा प्रकाशयत्येकः कृत्स्नं लोकमिमं रविः।क्षेत्...,यद्यपि आत्मा चेतना के साथ जिस शरीर में रहती है...
520,14,3,मम योनिर्महद्ब्रह्म तस्मिन् गर्भं दधाम्यहम्।सं...,जैसा कि 7वें और 8वें अध्याय में वर्णन किया गया...
521,14,4,सर्वयोनिषु कौन्तेय मूर्तयः सम्भवन्ति याः।तासां...,जैसा कि 7वें और 8वें अध्याय में वर्णन किया गया...
656,18,40,न तदस्ति पृथिव्यां वा दिवि देवेषु वा पुनः।सत्त...,श्वेताश्वतरोपनिषद् में माया शक्ति का विस्तारित...


## Whole

In [40]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [41]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [42]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
307,8,3,श्री भगवानुवाच\n\nअक्षरं ब्रह्म परमं स्वभावोऽध...,श्रीकृष्ण कहते हैं कि परम सत्ता को ब्रह्म कहा...
560,15,16,द्वाविमौ पुरुषौ लोके क्षरश्चाक्षर एव च।क्षरः स...,लौकिक क्षेत्र में माया जीवात्मा को भौतिक देह क...
561,15,17,उत्तमः पुरुषस्त्वन्यः परमात्मेत्युदाहृतः।यो लो...,संसार और आत्मा का निरूपण करने के पश्चात अब श्र...
